In [ ]:
import pandas as pd

khach_hang = pd.read_csv("../data_clean/customer_info_clean.csv")
hoa_don = pd.read_csv("../data_clean/invoice_header_clean.csv")
chi_tiet_hd = pd.read_csv("../data_clean/invoice_detail_clean.csv")

du_lieu = chi_tiet_hd.merge(hoa_don, on="invoice_id", how="left")

du_lieu = du_lieu.merge(
    khach_hang[['customer_id', 'region']],
    on="customer_id",
    how="left"
)



## Yêu cầu 1: Tính doanh thu theo từng sản phẩm

In [ ]:
du_lieu['doanh_thu'] = du_lieu['quantity'] * du_lieu['unit_price']

doanh_thu_theo_san_pham = (
    du_lieu
    .groupby('product_name')['doanh_thu']
    .sum()
    .reset_index()
    .sort_values(by='doanh_thu', ascending=False)
)

doanh_thu_theo_san_pham.head()


,product_name,doanh_thu
2,Kem Danh Rang,1137000.0
5,Sua Tuoi,974000.0
3,Mi Goi,941000.0
0,Ban Mi,880000.0
4,Nuoc Ngot,648000.0


## Yêu cầu 2: Doanh thu trung bình theo khu vực

In [3]:
doanh_thu_tb_khu_vuc = (
    du_lieu
    .groupby('region')['doanh_thu']
    .mean()
    .reset_index()
    .sort_values(by='doanh_thu', ascending=False)
)

doanh_thu_tb_khu_vuc

,region,doanh_thu
3,Trung,72076.923077
1,Nam,62846.153846
0,Bắc,59511.111111
2,Nan,42941.176471


## Yêu cầu 3: Sản phẩm có doanh thu cao nhất

In [4]:
san_pham_doanh_thu_cao_nhat = doanh_thu_theo_san_pham.iloc[0]

san_pham_doanh_thu_cao_nhat

product_name    Kem Danh Rang
doanh_thu           1137000.0
Name: 2, dtype: object

## Yêu cầu 4: Số lượng đơn hàng theo tháng

In [ ]:
du_lieu['date'] = pd.to_datetime(
    du_lieu['date'],
    format='%Y-%m-%d',
    errors='coerce'
)

du_lieu = du_lieu.dropna(subset=['date'])

du_lieu['thang'] = du_lieu['date'].dt.to_period('M')

so_don_hang_theo_thang = (
    du_lieu
    .groupby('thang')['invoice_id']
    .nunique()
    .reset_index(name='so_don_hang')
)
so_don_hang_theo_thang


,thang,so_don_hang
0,2024-01,11
1,2024-02,8
2,2024-03,5
3,2024-04,6


## Yêu cầu 5: Nhóm sản phẩm có doanh thu thấp

In [11]:
san_pham_doanh_thu_thap = (
    doanh_thu_theo_san_pham
    .sort_values(by='doanh_thu', ascending=True)
    .head(5)
)

san_pham_doanh_thu_thap


,product_name,doanh_thu
1,Dau An,582000.0
4,Nuoc Ngot,648000.0
0,Ban Mi,880000.0
3,Mi Goi,941000.0
5,Sua Tuoi,974000.0
